In [ ]:
import pickle,os,csv,sys, math
import pandas as pd
import matplotlib
matplotlib.use("AGG")
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

In [ ]:
# get ATC codes
dbdf = pd.read_excel('Drugbank050120.xlsx')
d2atc = defaultdict(set)
for (d,a_str) in zip(dbdf.drugbank_id,dbdf.atc_codes):
        #print((d,a_str))
        #print(type(a_str))
        if type(a_str) == type(1.0):
                continue
        if '|' in a_str:
                atc_codes = a_str.split('|')
        else:
                atc_codes = [a_str]
        # print(atc_codes)
        for a in atc_codes:
                keep_a = a[0]
                d2atc[d].add(keep_a)

In [ ]:
# load results of PathFX
f = 'Supplemental File 1_ SCZ_PSCZ_MDD_RD_UD_BPDv3.xlsx'
rdf = pd.read_excel(f)
r_dbids = set(rdf.DrugBankID)

In [ ]:
# map ATC codes to colors, drugs to colors, then make into dataframe
pfx_atc = dict([x for x in d2atc.items() if x[0] in r_dbids])
all_atc = sorted(set([x for aset in pfx_atc.values() for x in aset]))
print(all_atc)
atc_colors =['red','coral','peru','darkorange','gold','yellowgreen','green','lightseagreen','dodgerblue','slateblue','mediumorchid','violet','hotpink','lightpink']
atc_to_color = dict(zip(all_atc,atc_colors))
drug_to_color = dict([(d,atc_to_color[ac]) if len(ac_set)==1 else (d,'black') for (d,ac_set) in pfx_atc.items() for ac in ac_set]) # mixed class drugs in black
(dnames,dcolors) = zip(*[(k,v) for (k,v) in drug_to_color.items()])
rc_df = pd.DataFrame.from_dict({'Drugs': dnames,'ATC Codes': dcolors})
rc_df = rc_df.set_index('Drugs')

['A', 'B', 'C', 'D', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'V']


In [ ]:
# bar chart of all ATC codes
atc_count = defaultdict(int)
for (d,aset) in pfx_atc.items():
        for acode in aset:
                atc_count[acode]+=1

In [ ]:
# plot
(alabels,acounts) = zip(*sorted(atc_count.items()))
fig,ax = plt.subplots()
y_pos = np.arange(len(acounts))
bar_colors = [c for (a,c) in sorted(atc_to_color.items())]
hbars = ax.barh(y_pos,acounts,align='center',color=bar_colors)
ax.set_yticks(y_pos)
ax.set_yticklabels(alabels, fontsize=14)
# old school way to force bar labels
for (yp,ac) in zip(y_pos,acounts):
        n=ax.text(ac+5, yp, str(ac), fontsize=14)
ax.set_title('Number of active ingredients \nper level-1 ATC code', fontsize=14)
ax.set_ylim([-1,14])
ax.get_xaxis().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.invert_yaxis()  # labels read top-to-bottom
plt.savefig('Num_active_ingred_level1_atc_codes.png',format='png')